In [1]:
# So what questions do I have so far?
# 1. Should I use rho(r) to find r_s
# 2. Should I use V(r) to find V_200 and C_200
# 3. If 1 and 2 then I get two values for r_s
# 4. it seems that r_s obtained from rho(r) is more presize 
# 5. Should I find r_200 just from rho_enc (probably not)
# 6. should I find M_200 from mass profile from paper or by using mass_enc (assume paper)

In [2]:
import pynbody as pyn
import scipy
import numpy
from matplotlib import pylab as plt
from scipy.optimize import curve_fit as fit
s = pyn.load('testdata/g15784.lr.01024.gz')
s.physical_units()
h = s.halos()
rho_crit = pyn.analysis.cosmology.rho_crit(s)*200
H = float(pyn.analysis.cosmology.H(s))

In [3]:
tmass = h[1]['mass'].sum()

In [4]:
pyn.analysis.angmom.faceon(h[1])

In [5]:
p = pyn.analysis.profile.Profile(h[1], min = 0.01, max = 50, ndim = 3, type = 'log', nbins = 100)

/home/tengiz/.local/lib/python3.6/site-packages/pynbody/array.py:865: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return self.base[self._reexpress_index(item)]


In [6]:
radii = p['rbins']
vel = p['v_circ']
den = p['density']

pynbody.analysis.profile : Profile v_circ -- this routine assumes the disk is in the x-y plane


In [7]:
# pISO profile function

initial_guess = [10**10, 0.01]
def rho_pISO(r, rho_s,r_s): 
    # r200 is the radius inside of which the average halo (bg paper)
    #density is 200 times the critical density of the universe.
    return rho_s/(1+(r/r_s)**2)
param, covar = fit(rho_pISO, radii, den, p0 = initial_guess)

#new list for r/r_s
x = []
for r in radii:
    x.append(r/param[1])

def M_pISO(r):
    # we define M200 and N200 as the mass and the number of particles within r200 (Maccio 2008)
    return 4*numpy.pi*param[0]*(param[1]**3)*(r-numpy.arctan(r))**0.5
    
def V_pISO(r, C_200, V200):
     return V200*((1-numpy.arctan(r)/r)/(1-numpy.arctan(C_200)/C_200))**0.5

param1, covar1 = fit(V_pISO, x, vel)

In [8]:
# den plot

# fig, ax = plt.subplots() 
# ax.semilogy(radii, den, 'g--')
# ax.semilogy(radii, rho_pISO(radii, *param), 'r-')
# ax.grid()
# ax.legend(('data','fit'))
# ax.set_title('data den vs r')
# ax.set_xlabel(radii.units)
# ax.set_ylabel(den.units)
# plt.show()

In [9]:
# vel plot
# fig1, ax1 = plt.subplots()
# ax1.plot(x, vel, 'g--')
# ax1.plot(x, V_pISO(x, *param1), 'r-')
# ax1.grid()
# ax1.legend(('data','fit'))
# ax1.set_title('data vel vs x')
# ax1.set_xlabel('$r_{s}$')
# ax1.set_ylabel(vel.units)
# plt.show()


In [10]:
# M_pISO(param1[0]*param[1]) 

In [15]:
import DM_Profiles
a,b = DM_Profiles.pISO(radii, den, vel)

In [16]:
a

431239423222.8395

In [17]:
b

577.0267937171411